# Finetuning


In [ ]:
!pip install optuna
!pip uninstall transformers -y
!pip install transformers
!pip uninstall bitsandbytes -y
!pip install python-dotenv
!pip install datasets
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.1 MB/s eta 0:00:00
Found existing installation: transformers 4.48.3
Uninstalling transformers-4.48.3:
  Successfully uninstalled transformers-4.48.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s

In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from peft import(
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    PeftModel
)
from dotenv import load_dotenv
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
from huggingface_hub import login
#login('')

load_dotenv()

False

In [ ]:
model_version : str   = '3B'
model_id      : str   = f'meta-llama/Llama-3.2-{model_version}'
#dataset_id    : str   = 'istat-ai/hs_dataset'
num_labels    : int   = 4

lora_r        : int   = 16
lora_alpha    : int   = 32
target_modules: list  =['down_proj', 'gate_proj', 'o_proj', 'v_proj', 'up_proj', 'q_proj', 'k_proj']
 #['q_proj', 'k_proj', 'v_proj', 'o_proj']
lora_dropout  : float = 0.05
lora_bias     : str   = 'none'

output_dir    : str   = 'LLama3-3B-finetuning-italian-version'
epochs        : int   = 10
learn_rate    : float = 2e-5
scheduler     : str   = 'linear'
train_bs      : int   = 16
eval_bs       : int   = 16
ga_steps      : int   = 2
decay         : float = 0.01
warmup        : float = 0.1
log_steps     : int   = 10
eval_strategy : str   = 'epoch'
save_strategy : str   = 'epoch'
fp16          : bool  = True
bf16          : bool = False

load_best     : bool  = True
#report_to     : list  = []
log_level     : str   = 'warning'

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=target_modules,
    lora_dropout=lora_dropout,
    bias=lora_bias,
    task_type='SEQ_CLS'
)
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    quantization_config=quant_config
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    add_prefix_space=True
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.pad_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-3B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/train_sample.csv')
X_val = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/val_sample.csv')
#X_test = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/sample_test.csv')
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
#print(f"Test set size: {len(X_test)}")
def tokenize(example):
    return tokenizer(example["italian text"], padding=True, truncation=True, max_length=tokenizer.model_max_length)
train = Dataset.from_pandas(X_train)
#test = Dataset.from_pandas(X_test)
validation = Dataset.from_pandas(X_val)
tokenized_train = train.map(tokenize, batched=True)
#tokenized_test=test.map(tokenize, batched=True)
tokenized_val=validation.map(tokenize, batched=True)
id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {'Depression': 0, 'Normal': 1,'Anxiety': 2, 'Suicidal':3}


Training set size: 3200
Validation set size: 600


Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
tokenized_train = tokenized_train.remove_columns(["text", "italian text"])
tokenized_val = tokenized_val.remove_columns(["text", "italian text"])
#tokenized_test = tokenized_test.remove_columns(["text", "italian text"])  # If using test data


In [ ]:
def model_init():
  model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=num_labels,quantization_config=quant_config)
  model = prepare_model_for_kbit_training(model)
  model = get_peft_model(model, lora_config)
  tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    add_prefix_space=True
)
  tokenizer.pad_token_id = tokenizer.eos_token_id
  tokenizer.pad_token = tokenizer.eos_token

  model.config.pad_token_id = tokenizer.pad_token_id
  return model

In [ ]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-6, 5e-5, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.1),  # Regularization
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4, 8]),  # Accumulation steps
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type", ["linear", "cosine", "constant", "cosine_with_restarts"]),  # Scheduler
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.0, 0.1),  # Warmup ratio
    }


In [ ]:
def compute_objective(metrics):
    print(metrics)
    return metrics["eval_f1_macro"]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer,   DataCollatorWithPadding,Trainer, TrainingArguments

from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import numpy as np

id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {v: k for k, v in id2label.items()}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)

    f1 = f1_score(labels, predictions, average='macro')

    cm = confusion_matrix(labels, predictions)

    cm_labels = np.array([id2label[i] for i in range(len(id2label))])
    cm_with_labels = pd.DataFrame(cm, index=cm_labels, columns=cm_labels)

    class_report = classification_report(labels, predictions, target_names=[id2label[i] for i in range(len(id2label))])

    print("Confusion Matrix:")
    print(cm_with_labels)
    print("\nClassification Report:")
    print(class_report)

    return {
        'accuracy': accuracy,
        'f1_macro': f1,
    }

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="Llama3-hyperparameter-search",
    num_train_epochs=2,
    per_device_train_batch_size=train_bs,
    per_device_eval_batch_size=eval_bs,
    metric_for_best_model="f1_macro",
    logging_dir='./logs',
    logging_steps=log_steps,
    eval_strategy=eval_strategy,
    save_strategy=save_strategy,
    fp16=fp16,
    load_best_model_at_end=load_best,
    log_level=log_level,
)
trainer = Trainer(
    model=None,
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    data_collator=data_collator)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-3B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=30,
    compute_objective=compute_objective,
)

[I 2025-02-26 20:29:51,348] A new study created in memory with name: no-name-1b13c92b-4f5c-421e-b1f8-aee2398da898
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-3B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,1.366200,1.112286,0.538333,0.538791
2,0.921500,0.969239,0.585000,0.585034


Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression          60      15       14        61
Normal               8     108        7        27
Anxiety             29      27       73        21
Suicidal            42      20        6        82

Classification Report:
              precision    recall  f1-score   support

  Depression       0.43      0.40      0.42       150
      Normal       0.64      0.72      0.68       150
     Anxiety       0.73      0.49      0.58       150
    Suicidal       0.43      0.55      0.48       150

    accuracy                           0.54       600
   macro avg       0.56      0.54      0.54       600
weighted avg       0.56      0.54      0.54       600

{'eval_loss': 1.1122857332229614, 'eval_accuracy': 0.5383333333333333, 'eval_f1_macro': 0.5387908324792743}


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression          69       9       19        53
Normal               8     114       12        16
Anxiety             26      22       91        11
Suicidal            47      16       10        77

Classification Report:
              precision    recall  f1-score   support

  Depression       0.46      0.46      0.46       150
      Normal       0.71      0.76      0.73       150
     Anxiety       0.69      0.61      0.65       150
    Suicidal       0.49      0.51      0.50       150

    accuracy                           0.58       600
   macro avg       0.59      0.58      0.59       600
weighted avg       0.59      0.58      0.59       600

{'eval_loss': 0.9692385792732239, 'eval_accuracy': 0.585, 'eval_f1_macro': 0.5850344266195482}


[I 2025-02-27 01:21:53,786] Trial 29 finished with value: 0.5850344266195482 and parameters: {'learning_rate': 3.920837263439173e-05, 'weight_decay': 0.03994527999788104, 'gradient_accumulation_steps': 8, 'lr_scheduler_type': 'cosine', 'warmup_ratio': 0.002464289241526231}. Best is trial 9 with value: 0.7956131855133967.


In [ ]:
print("Best Trial:")
print("Params:", best_trial.hyperparameters)

Best Trial:
Params: {'learning_rate': 4.163096456488191e-05, 'weight_decay': 0.02813545381523279, 'gradient_accumulation_steps': 1, 'lr_scheduler_type': 'cosine', 'warmup_ratio': 0.011919895317629037}


In [ ]:
import os

# Change the current working directory
os.chdir("/content/drive/MyDrive/Italian thesis/")

# Verify the current working directory
print("Current Working Directory:", os.getcwd())

Current Working Directory: /content/drive/MyDrive/Italian thesis


In [ ]:
import json
json_file_path = "llama3_hyperparameters.json"

# Write hyperparameters to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(best_trial.hyperparameters, json_file, indent=4)

print(f"Hyperparameters saved to {json_file_path}")

Hyperparameters saved to llama3_hyperparameters.json


# Training

In [ ]:
!pip uninstall transformers -y
!pip install transformers
!pip uninstall bitsandbytes -y
!pip install python-dotenv
!pip install datasets
!pip install -U bitsandbytes


Found existing installation: transformers 4.48.3
Uninstalling transformers-4.48.3:
  Successfully uninstalled transformers-4.48.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s 

In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from peft import(
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    PeftModel
)
from dotenv import load_dotenv
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
from huggingface_hub import login
#login('')

load_dotenv()

False

In [ ]:
model_version : str   = '3B'
model_id      : str   = f'meta-llama/Llama-3.2-{model_version}'
#dataset_id    : str   = 'istat-ai/hs_dataset'
num_labels    : int   = 4

lora_r        : int   = 16
lora_alpha    : int   = 32
target_modules: list  =['down_proj', 'gate_proj', 'o_proj', 'v_proj', 'up_proj', 'q_proj', 'k_proj']
 #['q_proj', 'k_proj', 'v_proj', 'o_proj']
lora_dropout  : float = 0.05
lora_bias     : str   = 'none'

output_dir    : str   = 'LLama3-3B-finetuning-italian-version'
epochs        : int   = 7
learn_rate    : float = 4.163096456488191e-05
scheduler     : str   = 'cosine'
train_bs      : int   = 16
eval_bs       : int   = 16
ga_steps      : int   = 1
decay         : float = 0.02813545381523279
warmup        : float = 0.011919895317629037
log_steps     : int   = 10
eval_strategy : str   = 'epoch'
save_strategy : str   = 'epoch'
fp16          : bool  = True
bf16          : bool = False

load_best     : bool  = True
#report_to     : list  = []
log_level     : str   = 'warning'

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=target_modules,
    lora_dropout=lora_dropout,
    bias=lora_bias,
    task_type='SEQ_CLS'
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    quantization_config=quant_config
)
# base_model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=num_labels,quantization_config=quant_config)
# model = PeftModel.from_pretrained(base_model, "msab97/LLama3-1B-finetuning")

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    add_prefix_space=True
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.pad_token_id


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-3B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
model.print_trainable_parameters()

trainable params: 24,326,144 || all params: 3,237,088,256 || trainable%: 0.7515


In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/train.csv')
X_val = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/val.csv')
X_test = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/test.csv')

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")
X_train.head()

Training set size: 11760
Validation set size: 1470
Test set size: 1470


,text,label,italian text
0,Im at the point of the semester where Im so ti...,3,Sono arrivato al punto del semestre in cui son...
1,I could do it I want to do it I have what I ne...,3,Potrei farlo Voglio farlo Ho ciò che mi serve ...
2,I cannot imagine anyone wanting to be in a rel...,3,Non riesco a immaginare qualcuno che voglia av...
3,How I am supposed to live if I cannot accept t...,3,Come dovrei vivere se non riesco ad accettare ...
4,maybe i know but how could you know,1,forse lo so ma come potresti saperlo?


In [ ]:
X_train = X_train.dropna(subset=['italian text'])


In [ ]:

def tokenize(example):
    return tokenizer(example["italian text"], padding=True, truncation=True, max_length=tokenizer.model_max_length)
train = Dataset.from_pandas(X_train)
test = Dataset.from_pandas(X_test)
validation = Dataset.from_pandas(X_val)
tokenized_train = train.map(tokenize, batched=True, remove_columns=["text", "italian text"])
tokenized_test = test.map(tokenize, batched=True, remove_columns=["text", "italian text"])
tokenized_val = validation.map(tokenize, batched=True, remove_columns=["text", "italian text"])
id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {'Depression': 0, 'Normal': 1,'Anxiety': 2, 'Suicidal':3}


Map:   0%|          | 0/11760 [00:00<?, ? examples/s]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer,   DataCollatorWithPadding,Trainer, TrainingArguments

from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import pandas as pd

# Define your class labels
id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {v: k for k, v in id2label.items()}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute F1 score (macro-average)
    f1 = f1_score(labels, predictions, average='macro')

    # Compute precision and recall (macro-average)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')

    # Compute confusion matrix
    cm = confusion_matrix(labels, predictions)

    # Convert confusion matrix from class IDs to labels
    cm_labels = np.array([id2label[i] for i in range(len(id2label))])
    cm_with_labels = pd.DataFrame(cm, index=cm_labels, columns=cm_labels)

    # Generate the classification report with labels
    class_report = classification_report(labels, predictions, target_names=[id2label[i] for i in range(len(id2label))])

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(cm_with_labels)
    print("\nClassification Report:")
    print(class_report)

    return {
        'accuracy': accuracy,
        'f1_macro': f1,
        'precision_macro': precision,
        'recall_macro': recall,
    }


In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=epochs,
    learning_rate=learn_rate,
    lr_scheduler_type=scheduler,
    per_device_train_batch_size=train_bs,
    per_device_eval_batch_size=eval_bs,
    gradient_accumulation_steps=ga_steps,
    warmup_ratio=warmup,
    metric_for_best_model="f1_macro",
    weight_decay=decay,
    logging_dir='./logs',
    logging_steps=log_steps,
    eval_strategy=eval_strategy,
    save_strategy=save_strategy,
    fp16=fp16,
    load_best_model_at_end=load_best,
    log_level=log_level,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,
    early_stopping_threshold=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[early_stopping]
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print('Results before finetuning:')
print('-'*15)

print(results)

Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression         281      15        6        98
Normal              10     382        3         5
Anxiety              8       9      251         2
Suicidal            64      13        4       319

Classification Report:
              precision    recall  f1-score   support

  Depression       0.77      0.70      0.74       400
      Normal       0.91      0.95      0.93       400
     Anxiety       0.95      0.93      0.94       270
    Suicidal       0.75      0.80      0.77       400

    accuracy                           0.84      1470
   macro avg       0.85      0.85      0.85      1470
weighted avg       0.84      0.84      0.84      1470

Results before finetuning:
---------------
{'eval_loss': 0.47628432512283325, 'eval_model_preparation_time': 0.0079, 'eval_accuracy': 0.8387755102040816, 'eval_f1_macro': 0.8459394674949097, 'eval_precision_macro': 0.8472288150647657, 'eval_recall_macro': 0.846157407407407

In [ ]:
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1 Macro,Precision Macro,Recall Macro
1,0.507900,0.459383,0.006800,0.805442,0.810920,0.807135,0.817338
2,0.237500,0.445845,0.006800,0.824490,0.830553,0.843509,0.835440
3,0.235100,0.581632,0.006800,0.808163,0.816736,0.818971,0.817731
4,0.062700,0.954647,0.006800,0.815646,0.825300,0.823752,0.827014


Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression         258      12       27       103
Normal               5     373       10        12
Anxiety              5       6      257         2
Suicidal            84      16        4       296

Classification Report:
              precision    recall  f1-score   support

  Depression       0.73      0.65      0.69       400
      Normal       0.92      0.93      0.92       400
     Anxiety       0.86      0.95      0.90       270
    Suicidal       0.72      0.74      0.73       400

    accuracy                           0.81      1470
   macro avg       0.81      0.82      0.81      1470
weighted avg       0.80      0.81      0.80      1470



/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression         228      15        9       148
Normal               2     387        2         9
Anxiety              3       6      259         2
Suicidal            39      22        1       338

Classification Report:
              precision    recall  f1-score   support

  Depression       0.84      0.57      0.68       400
      Normal       0.90      0.97      0.93       400
     Anxiety       0.96      0.96      0.96       270
    Suicidal       0.68      0.84      0.75       400

    accuracy                           0.82      1470
   macro avg       0.84      0.84      0.83      1470
weighted avg       0.83      0.82      0.82      1470



/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression         293      17        9        81
Normal               5     388        0         7
Anxiety              7      12      250         1
Suicidal           120      21        2       257

Classification Report:
              precision    recall  f1-score   support

  Depression       0.69      0.73      0.71       400
      Normal       0.89      0.97      0.93       400
     Anxiety       0.96      0.93      0.94       270
    Suicidal       0.74      0.64      0.69       400

    accuracy                           0.81      1470
   macro avg       0.82      0.82      0.82      1470
weighted avg       0.81      0.81      0.81      1470



/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression         274      10       13       103
Normal              10     377        4         9
Anxiety              5       6      258         1
Suicidal            99       9        2       290

Classification Report:
              precision    recall  f1-score   support

  Depression       0.71      0.69      0.70       400
      Normal       0.94      0.94      0.94       400
     Anxiety       0.93      0.96      0.94       270
    Suicidal       0.72      0.72      0.72       400

    accuracy                           0.82      1470
   macro avg       0.82      0.83      0.83      1470
weighted avg       0.81      0.82      0.81      1470



TrainOutput(global_step=2940, training_loss=0.30534308409305655, metrics={'train_runtime': 5059.3494, 'train_samples_per_second': 16.269, 'train_steps_per_second': 1.017, 'total_flos': 3.144798981750497e+17, 'train_loss': 0.30534308409305655, 'epoch': 4.0})

In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print('Results after finetuning:')
print('-'*15)
print(results)

Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression         221      20       14       145
Normal               3     383        4        10
Anxiety             12      12      241         5
Suicidal            33      16        5       346

Classification Report:
              precision    recall  f1-score   support

  Depression       0.82      0.55      0.66       400
      Normal       0.89      0.96      0.92       400
     Anxiety       0.91      0.89      0.90       270
    Suicidal       0.68      0.86      0.76       400

    accuracy                           0.81      1470
   macro avg       0.83      0.82      0.81      1470
weighted avg       0.82      0.81      0.80      1470

Results after finetuning:
---------------
{'eval_loss': 0.5150622129440308, 'eval_model_preparation_time': 0.0068, 'eval_accuracy': 0.810204081632653, 'eval_f1_macro': 0.8122218031311679, 'eval_precision_macro': 0.8267164207647879, 'eval_recall_macro': 0.8168981481481481, 

In [ ]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/msab97/LLama3-3B-finetuning-italian-version/commit/a0d2b33a3d111ee9c717305c14fd85a723bb34e3', commit_message='End of training', commit_description='', oid='a0d2b33a3d111ee9c717305c14fd85a723bb34e3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/msab97/LLama3-3B-finetuning-italian-version', endpoint='https://huggingface.co', repo_type='model', repo_id='msab97/LLama3-3B-finetuning-italian-version'), pr_revision=None, pr_num=None)

# Predictions

In [ ]:
!pip install datasets
!pip uninstall bitsandbytes -y
!pip install datasets
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
model_version : str   = '3B'
model_id      : str   = f'meta-llama/Llama-3.2-{model_version}'
#dataset_id    : str   = 'istat-ai/hs_dataset'
num_labels    : int   = 4

lora_r        : int   = 16
lora_alpha    : int   = 32
target_modules: list  =['down_proj', 'gate_proj', 'o_proj', 'v_proj', 'up_proj', 'q_proj', 'k_proj']
 #['q_proj', 'k_proj', 'v_proj', 'o_proj']
lora_dropout  : float = 0.05
lora_bias     : str   = 'none'

output_dir    : str   = 'LLama3-3B-finetuning-italian-version'
epochs        : int   = 7
learn_rate    : float = 3.9922302694761015e-05
scheduler     : str   = 'cosine'
train_bs      : int   = 16
eval_bs       : int   = 16
ga_steps      : int   = 1
decay         : float = 0.029022800303053556
warmup        : float = 0.0960656012161834
log_steps     : int   = 10
eval_strategy : str   = 'epoch'
save_strategy : str   = 'epoch'
fp16          : bool  = True
bf16          : bool = False

load_best     : bool  = True
#report_to     : list  = []
log_level     : str   = 'warning'

In [ ]:


from peft import PeftModel
from transformers import AutoModelForSequenceClassification,BitsAndBytesConfig
from huggingface_hub import login
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
import torch

#login('')
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16)
base_model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=num_labels,quantization_config=quant_config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-3B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = PeftModel.from_pretrained(base_model, "msab97/LLama3-3B-finetuning-italian-version")

adapter_config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.4M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    add_prefix_space=True
)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.pad_token_id

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Italian thesis/Training dataset/italian_with_predictions.csv")

df = df[['testo','tweet_date']]

In [ ]:
len(df)

456000

In [ ]:
def tokenize(example):
    return tokenizer(example["testo"], padding=True, truncation=True, max_length=tokenizer.model_max_length)
data = Dataset.from_pandas(df)
tokenized_data=data.map(tokenize, batched=True)



Map:   0%|          | 0/456000 [00:00<?, ? examples/s]

In [ ]:
tokenized_data = tokenized_data.remove_columns(["testo", "tweet_date"])


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer,   DataCollatorWithPadding,Trainer, TrainingArguments

from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import numpy as np

id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {v: k for k, v in id2label.items()}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)

    f1 = f1_score(labels, predictions, average='macro')

    cm = confusion_matrix(labels, predictions)

    cm_labels = np.array([id2label[i] for i in range(len(id2label))])
    cm_with_labels = pd.DataFrame(cm, index=cm_labels, columns=cm_labels)

    class_report = classification_report(labels, predictions, target_names=[id2label[i] for i in range(len(id2label))])

    print("Confusion Matrix:")
    print(cm_with_labels)
    print("\nClassification Report:")
    print(class_report)

    return {
        'accuracy': accuracy,
        'f1_macro': f1,
    }

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_eval_batch_size=eval_bs,
    logging_dir='./logs',
    fp16=fp16,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)


In [16]:
# Run the prediction
predictions = trainer.predict(tokenized_data)

# The logits are in predictions.predictions
logits = torch.tensor(predictions.predictions)

# If you want to see them, you can simply print or store them
df['llama_logits'] = logits.tolist()  # Storing logits as a list in the dataframe (optional)

# Optionally, if you want to keep the predicted classes as well:
predicted_classes = torch.argmax(logits, dim=-1).numpy()
df['llama_prediction'] = predicted_classes


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: masabbah97 (aml_group) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [17]:
df.head()
df.to_csv("/content/drive/MyDrive/Italian thesis/Training dataset/italian_with_predictions_and_logits.csv", index=False)


In [18]:
df.head()

,testo,tweet_date,llama_logits,llama_prediction
0,l istruzione e la ricchezza posson essere sorg...,2020-01-31,"[-2.353515625, 4.1796875, -2.923828125, -2.949...",1
1,ce l abbiamo fatta tutti in pratica parco nord...,2020-01-31,"[0.2763671875, 3.181640625, -5.38671875, 0.232...",1
2,qualcuno poi mi spieghi tutto il credito che s...,2020-01-31,"[-0.0018262863159179688, 2.484375, -7.28125, -...",1
3,aforismi f duva mov stelle e chi lo rappresent...,2020-01-31,"[0.78662109375, 3.923828125, -7.43359375, 0.48...",1
4,di solito quando un prodotto ha bisogno di tan...,2020-01-31,"[-0.41552734375, 5.20703125, -7.1796875, -1.48...",1
